# Plugin Resource Utilities

> Utilities for analyzing plugin configurations and resource requirements

These utilities work with any plugin metadata that has a `config_schema` attribute, including `PluginMetadata` from `cjm-fasthtml-plugins`.

In [ ]:
#| default_exp utils.plugin_utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional

from cjm_fasthtml_resources.core.manager import PLUGIN_RESOURCE_CONFIG_KEYS

## Plugin Type Detection

Helper functions to determine if a plugin is local vs API-based.

In [ ]:
#| export
def is_local_plugin(plugin_meta) -> bool:
    """
    Check if a plugin is local (vs API-based).
    
    Args:
        plugin_meta: Plugin metadata with config_schema attribute
    
    Returns:
        True if plugin is local, False if API-based
    """
    if not hasattr(plugin_meta, 'config_schema') or not plugin_meta.config_schema:
        return False

    # Check if schema has an api_key property
    properties = plugin_meta.config_schema.get('properties', {})
    return 'api_key' not in properties

Example plugin metadata structure:

```python
# Local plugin (works with any metadata object with config_schema)
local_plugin = PluginMetadata(
    name="whisper",
    config_schema={
        "properties": {
            "model_id": {"type": "string"},
            "device": {"type": "string"}
        }
    }
)

# API-based plugin  
api_plugin = PluginMetadata(
    name="openai-whisper",
    config_schema={
        "properties": {
            "api_key": {"type": "string"},  # Presence of api_key indicates API-based
            "model": {"type": "string"}
        }
    }
)
```

**Compatibility**: These utilities are compatible with:
- `PluginMetadata` from `cjm-fasthtml-plugins`
- `SimplePluginRegistry` from `cjm-fasthtml-settings`
- `UnifiedPluginRegistry` from `cjm-fasthtml-plugins`
- Any custom object with a `config_schema` attribute

## GPU Detection

Helper functions to determine if a plugin will use GPU resources.

In [ ]:
#| export
def uses_gpu_device(plugin_config: Dict[str, Any]) -> bool:
    """
    Check if a plugin is configured to use GPU.
    
    Args:
        plugin_config: The plugin's current configuration
    
    Returns:
        True if plugin will use GPU, False otherwise
    """
    device = plugin_config.get('device', '').lower()

    # Check for explicit GPU indicators
    if device and device != 'cpu':
        # Could be 'cuda', 'gpu', 'cuda:0', 'auto', etc.
        if 'cuda' in device or 'gpu' in device or device.startswith('mps') or device == 'auto':
            return True

    return False

In [ ]:
# Example: Check various device configurations
configs = [
    {"device": "cuda"},
    {"device": "cuda:0"},
    {"device": "cpu"},
    {"device": "auto"},
    {"device": "mps"},
    {}  # No device specified
]

for config in configs:
    device = config.get("device", "(not specified)")
    uses_gpu = uses_gpu_device(config)
    print(f"Device '{device}': GPU={uses_gpu}")

Device 'cuda': GPU=True
Device 'cuda:0': GPU=True
Device 'cpu': GPU=False
Device 'auto': GPU=True
Device 'mps': GPU=True
Device '(not specified)': GPU=False


## Resource Identifier Extraction

Extract the plugin resource identifier (model ID, model path, etc.) from configuration.

In [ ]:
#| export
def get_plugin_resource_identifier(plugin_config: Dict[str, Any]) -> Optional[str]:
    """
    Extract the plugin resource identifier from plugin configuration.
    
    Checks common plugin resource configuration keys like 'resource_id', 'model_id', 
    'model', 'model_name', etc.
    
    Args:
        plugin_config: The plugin's configuration
    
    Returns:
        Plugin resource identifier string, or None if not found
    """
    for key in PLUGIN_RESOURCE_CONFIG_KEYS:
        if key in plugin_config:
            value = plugin_config[key]
            if value:
                return str(value)

    return None

In [ ]:
# Example: Extract resource identifiers from different configs
configs = [
    {"model_id": "whisper-large-v3", "device": "cuda"},
    {"model": "llama-3.1-8b", "device": "auto"},
    {"model_path": "/models/custom-model", "device": "cpu"},
    {"device": "cuda"}  # No resource ID
]

for config in configs:
    resource_id = get_plugin_resource_identifier(config)
    print(f"Config {config}: Resource ID = {resource_id}")

Config {'model_id': 'whisper-large-v3', 'device': 'cuda'}: Resource ID = whisper-large-v3
Config {'model': 'llama-3.1-8b', 'device': 'auto'}: Resource ID = llama-3.1-8b
Config {'model_path': '/models/custom-model', 'device': 'cpu'}: Resource ID = /models/custom-model
Config {'device': 'cuda'}: Resource ID = None


## Resource Comparison

Compare two plugin configurations to determine if they use the same resource.

In [ ]:
#| export
def compare_plugin_resources(
    config1: Dict[str, Any],
    config2: Dict[str, Any]
) -> bool:
    """
    Compare two plugin configurations to see if they use the same plugin resource.
    
    Args:
        config1: First plugin configuration
        config2: Second plugin configuration
    
    Returns:
        True if both configs specify the same plugin resource, False otherwise
    """
    resource1 = get_plugin_resource_identifier(config1)
    resource2 = get_plugin_resource_identifier(config2)

    # Both must have resource identifiers to compare
    if resource1 is None or resource2 is None:
        return False

    return resource1 == resource2

In [ ]:
# Example: Compare configurations
config_a = {"model_id": "whisper-large-v3", "device": "cuda"}
config_b = {"model_id": "whisper-large-v3", "device": "cpu"}  # Same resource, different device
config_c = {"model_id": "whisper-medium", "device": "cuda"}  # Different resource

print(f"A vs B (same resource): {compare_plugin_resources(config_a, config_b)}")
print(f"A vs C (different resource): {compare_plugin_resources(config_a, config_c)}")
print(f"B vs C (different resource): {compare_plugin_resources(config_b, config_c)}")

A vs B (same resource): True
A vs C (different resource): False
B vs C (different resource): False


## Combined Resource Requirements

Get comprehensive resource requirements for a plugin.

In [ ]:
#| export
def get_plugin_resource_requirements(
    plugin_id: str,
    plugin_registry,  # Plugin registry with get_plugin, load_plugin_config methods
    plugin_config: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Get resource requirements for a plugin.
    
    Args:
        plugin_id: Unique plugin ID
        plugin_registry: Plugin registry instance
        plugin_config: Optional plugin configuration
    
    Returns:
        Dictionary with resource requirement information:
        - is_local: Whether it's a local plugin
        - uses_gpu: Whether it uses GPU
        - plugin_resource: Resource identifier
        - device: Device configuration
    """
    plugin_meta = plugin_registry.get_plugin(plugin_id)

    if not plugin_meta:
        return {
            'is_local': False,
            'uses_gpu': False,
            'plugin_resource': None
        }

    is_local = is_local_plugin(plugin_meta)

    if not is_local:
        return {
            'is_local': False,
            'uses_gpu': False,
            'plugin_resource': None
        }

    # Load config if not provided
    if plugin_config is None:
        plugin_config = plugin_registry.load_plugin_config(plugin_id)

    uses_gpu = uses_gpu_device(plugin_config)
    resource = get_plugin_resource_identifier(plugin_config)

    return {
        'is_local': is_local,
        'uses_gpu': uses_gpu,
        'plugin_resource': resource,
        'device': plugin_config.get('device', 'unknown')
    }

This function is the main entry point for getting plugin resource information. It combines all the other utility functions to provide a complete picture of a plugin's resource requirements.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()